In [30]:
import math
import numpy as np
from os import listdir
import os.path
from os.path import isfile
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import soundfile
from xml.etree import ElementTree
from warnings import warn

In [25]:
def read_X(path_to_wav, frame_rate_hz):
    # scipy.io.wavfile is not able to read 24-bit data hence the need to change this alternative library
    samples, sample_rate = soundfile.read(path_to_wav)
    if sample_rate % frame_rate_hz != 0:
        raise ValueError('Sample rate ' + str(sample_rate) + ' % frame rate ' + str(frame_rate_hz) + ' != 0')
    samples_per_frame = int(sample_rate / frame_rate_hz)
    offset = 0
    # TODO subsampling
    X = []
    # Cut off last samples
    while offset <= len(samples) - samples_per_frame:
        X.append(samples[offset:offset + samples_per_frame])
        offset += samples_per_frame

    X = np.array(X)
    return X, X.size / sample_rate


# TODO check that this is perfectly in sync with the actual onset to learn the right thing.
def read_y(path_to_xml, frame_rate_hz, length_seconds):
    tree = ElementTree.parse(path_to_xml)
    root = tree.getroot()
    y = np.zeros(round(frame_rate_hz * length_seconds), dtype=np.int8)
    for root_child in root:
        if root_child.tag == 'transcription':
            for event in root_child:
                if event.tag != 'event':
                    raise ValueError('Unexpected XML element, expected event, got ' + event.tag)
                for event_child in event:
                    if event_child.tag == 'onsetSec':
                        index = int(float(event_child.text) * frame_rate_hz)
                        y[index] = 1
            break

    y = np.reshape(y, (-1, 1))
    return y

In [26]:
# TODO dataset einchecken, damit aenderungen zentral gemacht werden (anderes, privates repo)
frame_rate_hz = 100
data_dir = 'data/IDMT-SMT-GUITAR_V2/dataset2'
audio_dir = os.path.join(data_dir, 'audio')
annotation_dir = os.path.join(data_dir, 'annotation')
X = None
y = None
for wav_file in listdir(audio_dir):
    if wav_file.endswith('.wav'):
        X_part, length_seconds = read_X(os.path.join(audio_dir, wav_file), frame_rate_hz)
        path_to_xml = os.path.join(annotation_dir, wav_file.replace('.wav', '.xml'))
        if isfile(path_to_xml):
            y_part = read_y(path_to_xml, frame_rate_hz, length_seconds)
            if X_part.shape[0] != y_part.shape[0]:
                raise ValueError('X vs. y shape mismatch: ' + str(X_part.shape[0]) + ' != ' + str(y_part.shape[0]))
            # TODO is there a faster way than this vstacking?
            if X is None:
                X = X_part
            else:
                X = np.vstack((X, X_part))
            if y is None:
                y = y_part
            else:
                y = np.vstack((y, y_part))
        else:
            warn('No truth found for ' + wav_file + ', skipping file.')

print(X.shape)
print(y.shape)

D:\ProgramFiles\Anaconda3_64\lib\site-packages\ipykernel\__main__.py:26: UserWarning: No truth found for AR_Lick11_FN.wav, skipping file.
D:\ProgramFiles\Anaconda3_64\lib\site-packages\ipykernel\__main__.py:26: UserWarning: No truth found for AR_Lick11_KN.wav, skipping file.
D:\ProgramFiles\Anaconda3_64\lib\site-packages\ipykernel\__main__.py:26: UserWarning: No truth found for AR_Lick11_MN.wav, skipping file.
D:\ProgramFiles\Anaconda3_64\lib\site-packages\ipykernel\__main__.py:26: UserWarning: No truth found for FS_Lick11_FN.wav, skipping file.
D:\ProgramFiles\Anaconda3_64\lib\site-packages\ipykernel\__main__.py:26: UserWarning: No truth found for FS_Lick11_KN.wav, skipping file.
D:\ProgramFiles\Anaconda3_64\lib\site-packages\ipykernel\__main__.py:26: UserWarning: No truth found for FS_Lick11_MN.wav, skipping file.
D:\ProgramFiles\Anaconda3_64\lib\site-packages\ipykernel\__main__.py:26: UserWarning: No truth found for LP_Lick11_FN.wav, skipping file.
D:\ProgramFiles\Anaconda3_64\lib\s

(345916, 441)
(345916, 1)


In [27]:
print(y.shape)
y = y.ravel()
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(345916, 1)
(345916,)
(276732, 441)
(276732,)
(69184, 441)
(69184,)


In [31]:
# clf = RandomForestClassifier(n_jobs=-1)
clf = MLPClassifier()
clf.fit(X_train, y_train)
y_train_predicted = clf.predict(X_train)
y_test_predicted = clf.predict(X_test)

In [29]:
print(classification_report(y_train, y_train_predicted))
print(classification_report(y_test, y_test_predicted))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00    273663
          1       1.00      0.77      0.87      3069

avg / total       1.00      1.00      1.00    276732

             precision    recall  f1-score   support

          0       0.99      1.00      0.99     68464
          1       0.00      0.00      0.00       720

avg / total       0.98      0.99      0.98     69184



In [32]:
print(classification_report(y_train, y_train_predicted))
print(classification_report(y_test, y_test_predicted))

             precision    recall  f1-score   support

          0       0.99      1.00      0.99    273663
          1       0.97      0.01      0.02      3069

avg / total       0.99      0.99      0.98    276732

             precision    recall  f1-score   support

          0       0.99      1.00      0.99     68464
          1       0.00      0.00      0.00       720

avg / total       0.98      0.99      0.98     69184



- plot X vs y obs stimmt
- zu wenige daten?